# Imports

In [1]:
# Hakim
import numpy as np
import pandas as pd
from  sklearn.svm import *
from sklearn.metrics import f1_score, precision_recall_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay

# Data Preprocessing 

In [2]:
# Hakim
train_df=pd.read_csv('/kaggle/input/frauddetection-data/fraudTrain.csv')
test_df=pd.read_excel('/kaggle/input/frauddetection-data/fraudTest.xlsx')

In [3]:

train_df['category'] = train_df['category'].astype('category')
train_df['category_index'] = train_df['category'].cat.codes

train_df['merchant'] = train_df['merchant'].astype('category')
train_df['merchant_index'] = train_df['merchant'].cat.codes
train_df.head()


# TODO Tebry

,ID,Time,Card Number,merchant,category,Amount,firstName,lastName,trans_num,is_fraud,category_index,merchant_index
0,0,1/1/2019 0:00,2.700000e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,0b242abb623afc578575680df30655b9,0.0,9,514
1,1,1/1/2019 0:00,6.300000e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,1f76529f8574734946361c461b024d99,0.0,4,241
2,2,1/1/2019 0:00,3.890000e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,a1a22d70485983eac12b5b88dad1cf95,0.0,0,390
3,3,1/1/2019 0:01,3.530000e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,6b849c168bdad6f867558c3793159a81,0.0,2,360
4,4,1/1/2019 0:03,3.760000e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,a41d7549acf90789359a9aa5346dcb46,0.0,10,297


# Feature Engineering

In [4]:
# TODO Tebry

# Create a copy of the DataFrame to avoid modifying the original DataFrame
train_df_copy = train_df.copy()

# Convert the 'Time' column to datetime format
train_df_copy['Time'] = pd.to_datetime(train_df_copy['Time'])


# Calculate the average transaction amount of all transactions
average_amount_all = train_df_copy['Amount'].mean()

# Calculate the difference between current transaction amount and average of all transactions
train_df_copy['Amount_diff_avg'] = train_df_copy['Amount'] - average_amount_all

# Calculate the time difference between previous transactions for each card in hours
time_diff_seconds = train_df_copy['Time'].diff().dt.total_seconds() % (24 * 3600)
train_df_copy['Time_diff_prev_transaction'] = time_diff_seconds / 3600  

columns_to_drop = ['ID', 'Time', 'Card Number', 'merchant', 'category', 'firstName', 'lastName', 'trans_num']
train_df_copy.drop(columns=columns_to_drop, inplace=True)

train_df_copy.dropna(inplace=True)
train_df_copy.head()


,Amount,is_fraud,category_index,merchant_index,Amount_diff_avg,Time_diff_prev_transaction
1,107.23,0.0,4,241,36.973843,0.000000
2,220.11,0.0,0,390,149.853843,0.000000
3,45.00,0.0,2,360,-25.256157,0.016667
4,41.96,0.0,10,297,-28.296157,0.033333
5,94.63,0.0,2,607,24.373843,0.016667


In [5]:
y_train=train_df_copy['is_fraud']
# should work after tebry does the test_df
# y_test=test_df_copy['is_fraud']

In [6]:
x_train=train_df_copy.drop('is_fraud',axis=1)
# should work after tebry does the test_df
# x_test=test_df_copy.drop('is_fraud',axis=1)

In [7]:
y_train=np.array(y_train)
# y_test=np.array(y_test)

# Model Selection and Training

In [8]:
# TODO Hakim and Ahmad
# another option
# model=LinearSVC(max_iter=2500)
# model.fit(x_train.head(10000),y_train[:10000])


In [9]:
svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
# until now it is unnecessary to use al the data
svm.fit(x_train.head(10000), y_train[:10000])

SVC(gamma=0.5)

# Model Evaluation

In [10]:
svm.score(x_train.head(10000), y_train[:10000])

0.9997

In [11]:
x_train[10000:20000]

,Amount,category_index,merchant_index,Amount_diff_avg,Time_diff_prev_transaction
10001,9.46,9,661,-60.796157,0.016667
10002,61.05,2,208,-9.206157,0.016667
10003,84.23,2,181,13.973843,0.016667
10004,828.23,10,222,757.973843,0.000000
10005,88.61,12,630,18.353843,0.033333
...,...,...,...,...,...
19996,2.04,13,48,-68.216157,0.033333
19997,38.52,3,578,-31.736157,0.016667
19998,110.12,4,268,39.863843,0.000000
19999,52.22,2,321,-18.036157,0.016667


In [12]:
# TODO Watany me4 h3ml 7aga
y_pred = svm.predict(x_train[10000:20000])

f1 = f1_score(y_train[10000:20000], y_pred)
precision, recall, _ = precision_recall_curve(y_train[10000:20000], y_pred)
auc_prc = roc_auc_score(y_train[10000:20000], y_pred)

print("F1-score:", f1)
print("AUC-PRC:", auc_prc)

F1-score: 0.0
AUC-PRC: 0.5


# Model Tuning

In [13]:
# TODO Osama

# Deployment and Monitoring:

In [14]:
# TODO Saed